In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from numpy import cos
import math as math
from PIL import ImageOps

In [6]:
#Барицентрические координаты
def bar_coord(x,y,x0,y0,x1,y1,x2,y2):
    lambda0 = ((x - x2) * (y1 - y2) - (x1 - x2) * (y - y2)) / ((x0 - x2) * (y1 - y2) - (x1 - x2) * (y0 - y2))
    lambda1 = ((x0 - x2) * (y - y2) - (x - x2) * (y0 - y2)) / ((x0 - x2) * (y1 - y2) - (x1 - x2) * (y0 - y2))
    lambda2 = 1.0 - lambda0 - lambda1
    return lambda0, lambda1, lambda2

In [29]:
# Вычисление нормали
def norma (x0,y0,z0,x1,y1,z1,x2,y2,z2):
    n = [0] * 3
    n[0] = ((y1-y2) * (z1-z0)) - ((z1-z2) * (y1-y0))
    n[1] = ((x1-x2) * (z1-z0)) - ((z1-z2) * (x1-x0))
    n[2] = ((x1-x2) * (y1-y0)) - ((y1-y2) * (x1-x0))
    return n

In [30]:
def cos(n):
    cosA = n[2]/ math.sqrt(n[0]**2 + n[1]**2 + n[2]**2)
    return cosA

In [ ]:
def draw_triangle(x0,y0,x1,y1,x2,y2,img_mat,color, z0,z1,z2,z_buffer):
    xmin = int(min(x0, x1, x2)-1)
    ymin = int(min(y0, y1, y2)-1)
    xmax = int(max(x0, x1, x2)+1)
    ymax = int(max(y0, y1, y2)+1)

    if (xmin < 0): xmin = 0
    if (ymin < 0): ymin = 0
    for x in range(math.ceil(xmin), math.floor(xmax)):
        for y in range(math.ceil(ymin), math.floor(ymax)):
            lambda0, lambda1, lambda2 = bar_coord(x,y,x0,y0,x1,y1,x2,y2)
            if (lambda0 >=0 and lambda1 >=0 and lambda2 >=0): 
                z_display= lambda0*z0 + lambda1*z1 + lambda2*z2
                if(z_buffer[y,x] > z_display):
                     z_buffer[y,x] = z_display
                     img_mat[y,x]=color

In [45]:
#работа с файлом

file = open('model_1.obj', 'r')
v = []
f = []
img_mat = np.zeros((4000, 4000, 3), dtype = np.uint8)
z_buffer = np.full((4000,4000), np.inf, dtype = np.float32)

#считали вершины и вектора
for str in file:
    strs=str.split()
    if(strs[0] == 'v'):
         v.append((float(strs[1]),float(strs[2]),float(strs[3])))
    if(strs[0] == 'f'):
        f.append((int(strs[1].split('/')[0]),int(strs[2].split('/')[0]),int(strs[3].split('/')[0])))

#пробежались по вершинам которые надо соединить
for i in range(len(f)):
        
        x0 = (v[f[i][0]-1][0]* 20000 + 2000)
        y0 = (v[f[i][0]-1][1]* 20000 + 1200)
        z0 = (v[f[i][0]-1][2]* 20000 + 1200)
        x1 = (v[f[i][1]-1][0]* 20000 + 2000)
        y1 = (v[f[i][1]-1][1]* 20000 + 1200)
        z1 = (v[f[i][1]-1][2]* 20000 + 1200)
        x2 = (v[f[i][2]-1][0]* 20000 + 2000)
        y2 = (v[f[i][2]-1][1]* 20000 + 1200)
        z2 = (v[f[i][2]-1][2]* 20000 + 1200)

        n = norma(x0,y0,z0,x1,y1,z1,x2,y2,z2)
        cosA = cos(n)
        color = [-255*cosA, -255*cosA, -255*cosA]
        if (cosA < 0):
                draw_triangle(x0,y0,x1,y1,x2,y2, img_mat, color, z0, z1, z2, cosA, z_buffer)


img = Image.fromarray(img_mat, mode='RGB')
im = ImageOps.flip(img)
im.save("model_2_color.png")